# Amadeus Further Analysis

## Problem Statement and Background

In this project, we will attempt to discover the features behind the popular music of each generation. For instance, if Britney Spears, “Oops I Did It Again” made the charts in 2001, and The Beatles’ “Real Love” made the charts in 1996, we want to see what made the music popular back then – was it the timbre, audio quality, or lyrics? We will then attempt to build a model that is able to predict whether a song will be popular or not, and will also attempt to apply it to modern music. 

Questions: 
1. What features best predict the popularity of a song?
2. Do those features change with time?
3. Do different features change how long it takes for a song to become popular?

This is an interesting problem because it can be used to create music which is more likely to become popular. 

## Data Sources you Intend to Use?


The Dataset that we are using is the Million Song Dataset, which can be found here: http://labrosa.ee.columbia.edu/millionsong/. 

We are initially using the subset (10,000 songs) of the entire dataset, and once we are confident we have a substantial model, we will then expand the database and include all million songs, while running our model on an EC2 server.

## Data Joining/Cleaning You Did (4 points)
If data is being joined, describe the joining process and any problems with it - explain the metric used for fuzzy joins.

Explain how you will handle missing or duplicate keys. Describe the tools you used to examine/repair/clean the data.

If you found any statistical anomalies last time, explain how you plan to deal with them.


## Analysis Approach (3 points)
Describe what analysis you are doing: This will probably comprise:

- Featurization: Explain how you generated features from the raw data. e.g. thresholding to produce binary features, binning, tf-idf, multinomial -> multiple binary features (one-hot encoding). 
- Describe any value transformations you did, e.g. histogram normalization.
- Modeling: Which machine learning models did you try? Which do you plan to try in the future?
- Performance measurement: How will you evaluate your model and improve featurization etc.


### Featurization

Our featurization occurs in the script below. We first need to extract a set of important features from the hd5 dataformat, we chose duration, key, loudness, end of fade in, start of fade out, tempo, and time signiture. Most of our features from didn't require additional cleaning, as many are already in workable formats (int) we simply piped them into our various models.

### Modelling

We tried a number of machine learning models: k Nearest Neighbors, Random Forest, Linear Regression, and Logistic Regression. We had relatively poor performance with k Nearest Neighbors, and slightly better performance with Random Tree, Logistic Regression, and Linear Regression. We plan on further tuning the above models to improve performance, and additionally trying unsupervised machine learning to attempt to cluster our songs on measures besides year released (potential clusters could be genre, band gender, band age). For validation of these measures we'll also need to acquire a dataset which contains information about the bands in our dataset. 

### Performance Measurements

For our preliminary analysis, we aimed for completing one of our initial goals of determining when a song was released based on a song's features. Since our song release year range is about 100 years, we considered a successful classification as being within 5 years of the correct release year. We only considered the accuracy around this classification as being indicative of our model's performance. 


In [3]:
from setup import *
from sklearn import datasets

pp = pprint.PrettyPrinter(indent=2)

print_local = False # Allows output supression in ipynotebook. 

"""
Method to convert all hdf5 files into csv with 10,000 lines of format:
data:
key, mode, tempo, time_signature, loudness, *timbre*

target:
year

The indices of the two match up.

"""
def convert_to_csv():
    i = 0
    header = ['duration',
            'end_of_fade_in',
            'key',
            'key_confidence',
            'loudness',
            'start_of_fade_out',
            'tempo',
            'time_signature',
            'time_signature_confidence']
    #Include header which describes features extracted.
    data = [header]
    target = []
    count = 0


    for root, dirs, files in os.walk(msd_subset_data_path):
        files = glob.glob(os.path.join(root,'*.h5'))
        for f in files:
            local_print("Getting data from: " + str(f))
            with parser.File(f, 'r') as h5:
                year = get_year(h5)
                if year:
                    count +=1
                    target.append([year])
                    row = []
                    local_print("Getting duration...")
                    row += [get_analysis_property(h5,'duration')]
                    local_print("Getting End Fade...")
                    row += [get_analysis_property(h5,'end_of_fade_in')]
                    local_print("Getting Key...")
                    row += [get_analysis_property(h5,'key')]
                    row += [get_analysis_property(h5,'key_confidence')]
                    local_print("Getting Loudness...")
                    row += [get_analysis_property(h5,'loudness')]
                    local_print("Getting Start Fade Out...")
                    row += [get_analysis_property(h5,'start_of_fade_out')]
                    local_print("Getting Tempo...")
                    row += [get_analysis_property(h5,'tempo')]
                    local_print("Getting Time Signiture...")
                    row += [get_analysis_property(h5,'time_signature')]
                    row += [get_analysis_property(h5,'time_signature_confidence')]
                    local_print(str(len(row)) + " Features aquired.")
                    local_print(row)
                    # uncomment row below to get the timbre as well.
                    # row += [get_timbre(h5)]
                    # print row
                    data.append(row)
                    i+=1

    with open('data_no_timbre.csv', 'w+') as f:
        writer = csv.writer(f)
        writer.writerows(data)

    local_print(target)
    with open('target_no_timbre.csv', 'w+') as f:
        writer = csv.writer(f)
        writer.writerows(target)
        
    sample = data[1]
    return zip(sample, header)

def local_print(string):
    if print_local:
        print(string)

def get_timbre(h5):
    listy = []
    timbres = h5['analysis']['segments_timbre']
    print_local(len(timbres))
    timbres = np.array(timbres, dtype='f2')
    timbres = timbres.flatten()
    return list(timbres)

def get_analysis_property(h5, prop):
    to_return = h5['/analysis/songs'][prop][0]
    if to_return:
        return to_return
    else:
        return 0

def get_year(h5):
    to_return = h5['/musicbrainz/songs']['year'][0]
    if to_return:
        return to_return
    else:
        return 0


sample = convert_to_csv()
print(sample)


[[1969], [1982], [2007], [1984], [1986], [1987], [2004], [2004], [1985], [2005], [1997], [1961], [1972], [1964], [1992], [1997], [2007], [2003], [1995], [2004], [1992], [2002], [2006], [1995], [1974], [2009], [1997], [1977], [1986], [1966], [1985], [1995], [1997], [2000], [2003], [1992], [1990], [1985], [2000], [1997], [2004], [2000], [1963], [2001], [2005], [2002], [1990], [1999], [2005], [1990], [2001], [1961], [2006], [2000], [1995], [1991], [1997], [2005], [1985], [1971], [1994], [1971], [1999], [2009], [1995], [2007], [1999], [2001], [1990], [2009], [1991], [1997], [2005], [1988], [1984], [2001], [2001], [2010], [1989], [2004], [1971], [2006], [2009], [1991], [2009], [1986], [1988], [1998], [1998], [1994], [1999], [2010], [2005], [1981], [2000], [2009], [1996], [1983], [1997], [2010], [1986], [1988], [2004], [2005], [2006], [2007], [1989], [2003], [1996], [1999], [1995], [2001], [2008], [2001], [2007], [2001], [1989], [2000], [2003], [1995], [1999], [1986], [2009], [2001], [2000],

## Preliminary Results (6 Points)
Summarize the results you have so far:

Define suitable performance measures for your problem. Explain why they make sense, and what other measures you considered.
Give the results. These might include accuracy scores, ROC plots and AUC, or precision/recall plots, or results of hypothesis tests.
Describe any tuning that you did.
Explain any hypothesis tests you did. Be explicit about the null and alternative hypothesis. Be very clear about the test you used and how you used it. Include all the experiment details (between/within-subjects, degrees-of-freedom etc). Be frugal with tests. Do not try many tests and report the best results.
Use graphics! Please use visual presentation whenever possible. The next best option is a table. Try to avoid "inlining" important results.


In [4]:
from sklearn import cross_validation, ensemble, linear_model, feature_selection, neighbors
import csv
import numpy as np
import pdb

def main():
    print("Loading Data...")
    with open('data_no_timbre.csv','r') as f:
        reader = csv.reader(f)
        headers = next(reader)
    train = np.genfromtxt(open('data_no_timbre.csv','r'), delimiter=',', dtype='f8', skip_header=1)
    train[np.isnan(train)] = 0
    target = np.genfromtxt(open('target_no_timbre.csv','r'), delimiter=',', dtype='f8')

    print("Creating test and train data...")
    X_train, X_test, y_train, y_test = cross_validation.train_test_split(train, target, test_size=0.1, random_state=0)

    # pdb.set_trace()

    # kNN classifier
    print("Generating kNN...")
    knn = neighbors.KNeighborsClassifier() # default neighbors is 5,
    knn.fit(X_train, y_train);

    # Random Forest
    print("Generating Random Forest...")
    rf = ensemble.RandomForestClassifier(n_estimators=1000, n_jobs=8)
    rf.fit(X_train, y_train)

    # Linear Regression Model
    print("Generating Linear Regression...")
    lr = linear_model.LinearRegression(n_jobs=8)
    lr.fit(X_train, y_train)

    # Logistic Regression Model
    print("Generating Logistic Regression...")
    lr2 = linear_model.LogisticRegression()
    # lr2.fit(X_train, y_train)

    rfe = feature_selection.RFE(lr2, 2)
    rfe.fit(X_train, y_train)

    print("kNN Accuracy: ")
    model_accuracy(knn, X_test, y_test)
    print("Random Forest Accuracy: ")
    model_accuracy(rf, X_test, y_test)
    print("Linear Regression Accuracy: ")
    model_accuracy(lr, X_test, y_test)
    print("Logistic Regression Accuracy: ")
    model_accuracy(rfe, X_test, y_test)

def model_accuracy(model, test_set, test_target):
    guesses = model.predict(test_set)
    right = 0

    for counter, guess in enumerate(guesses):
        if abs(guess - test_target[counter]) < 5:
            right += 1

    print 'Number of close guesses are: ' + str(right)
    print 'Total accuracy = ' + str(right*1.0/len(test_target))



if __name__=="__main__":
    main()

Loading Data...
Creating test and train data...
Generating kNN...
Generating Random Forest...
Generating Linear Regression...
Generating Logistic Regression...
kNN Accuracy: 
Number of close guesses are: 119
Total accuracy = 0.254273504274
Random Forest Accuracy: 
Number of close guesses are: 192
Total accuracy = 0.410256410256
Linear Regression Accuracy: 
Number of close guesses are: 193
Total accuracy = 0.412393162393
Logistic Regression Accuracy: 
Number of close guesses are: 218
Total accuracy = 0.465811965812


## Final Analysis, any Obstacles (3 Points)
Describe the final analysis you plan to do:

- Scale: how much data will you use?
- Model complexity: What complexity of models will you use, this is relevant for models like clustering, factor models, Random Forests etc.
- What tools will you use?
- Estimate of processing time? You should be able to form an estimate of how much time you need on your chosen tools.
and outline any obstacles you foresee.



### Scale

We've been running our preliminary analysis on a subset of 10,000 songs (2.7 GB). We will be running our final analysis on 1,000,000 songs (~270GB). 

### Model Complexity

? Sid ?

### Tools

The tools we will be using will be mainly **scikitlearn** for our machine learning and modelling needs. We'll also likely use **d3** and **matplotlib** to create visualizations of the factors that we are considering. 

### Estimate of Processing Time

The above processing takes about 5 minutes to convert our hd5 data into a useable format, we expect that to scale to around 9 hours when we scale to the full dataset. The model generation time will likely also increase proportionately, from 2 minutes to around 3.5 hours. Our total processing time will likely be half a day for the 270 GB of data we expect to process. 